In [1]:
!nvidia-smi

Sun Jul 26 06:35:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install coremltools==4.0b1

     |████████████████████████████████| 604kB 6.7MB/s 
  Created wheel for coremltools: filename=coremltools-4.0b1-cp36-none-any.whl size=796442 sha256=683b2a1e5dd77ce7a13e6054c885a8c035135d3b1f302ac826ed9661154578d4
  Stored in directory: /root/.cache/pip/wheels/e6/2f/39/c7f309170b37269d937c226e676611092188765da1a7213a54
  Created wheel for attr: filename=attr-0.3.1-cp36-none-any.whl size=2459 sha256=dcced282f4dcdee3869bf0e5cf71d759ecea56572db65254a5f67c157e55cd99
  Stored in directory: /root/.cache/pip/wheels/f0/96/9b/1f8892a707d17095b5a6eab0275da9d39e68e03a26aee2e726
Successfully built coremltools attr


In [1]:
import shutil
import coremltools as ct

from fastai import *
from fastai.vision import *
from fastai.metrics import accuracy

import warnings
warnings.filterwarnings('ignore')

## Import Dataset from Kaggle

In [4]:
!wget -x --load-cookies cookies.txt 'https://www.kaggle.com/ayuraj/american-sign-language-dataset/download' -O data.zip
!unzip data.zip
shutil.rmtree('/content/asl/asl')

Streaming output truncated to the last 5000 lines.
  inflating: asl/s/s_60_rotate_10.jpeg  
  inflating: asl/s/s_60_rotate_2.jpeg  
  inflating: asl/s/s_60_rotate_3.jpeg  
  inflating: asl/s/s_60_rotate_4.jpeg  
  inflating: asl/s/s_60_rotate_5.jpeg  
  inflating: asl/s/s_60_rotate_6.jpeg  
  inflating: asl/s/s_60_rotate_7.jpeg  
  inflating: asl/s/s_60_rotate_8.jpeg  
  inflating: asl/s/s_60_rotate_9.jpeg  
  inflating: asl/s/s_61_rotate_1.jpeg  
  inflating: asl/s/s_61_rotate_10.jpeg  
  inflating: asl/s/s_61_rotate_2.jpeg  
  inflating: asl/s/s_61_rotate_3.jpeg  
  inflating: asl/s/s_61_rotate_4.jpeg  
  inflating: asl/s/s_61_rotate_5.jpeg  
  inflating: asl/s/s_61_rotate_6.jpeg  
  inflating: asl/s/s_61_rotate_7.jpeg  
  inflating: asl/s/s_61_rotate_8.jpeg  
  inflating: asl/s/s_61_rotate_9.jpeg  
  inflating: asl/s/s_62_rotate_1.jpeg  
  inflating: asl/s/s_62_rotate_10.jpeg  
  inflating: asl/s/s_62_rotate_2.jpeg  
  inflating: asl/s/s_62_rotate_3.jpeg  
  inflating: asl/s/s_62_ro

## Train Classifier

In [2]:
batch_size = 64
root = '/content/asl'

In [3]:
files, labels = [], []
for r, d, f in os.walk(root):
    for i in f:
        label = r.split('/')[-1]
        files.append(f'{label}/{i}')
        labels.append(label)
df = pd.DataFrame(list(zip(files, labels)), columns=['name', 'label'])
data = ImageDataBunch.from_df(root, df, size=400, bs=batch_size // 2).normalize(imagenet_stats)

In [7]:
classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(),
    nn.BatchNorm1d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.Dropout(p=0.25, inplace=False),
    nn.Linear(in_features=1280, out_features=512, bias=True),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=36, bias=True)
)

learn = cnn_learner(data, models.mobilenet_v2, metrics=accuracy, custom_head=classifier)
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.406323,0.177373,0.940490,03:16
1,0.189771,0.085553,0.969949,03:16
2,0.078676,0.024098,0.991696,03:15
3,0.035687,0.006994,0.997232,03:16
4,0.019773,0.003612,0.999011,03:15


## PyTorch --> CoreML

In [8]:
model = learn.model.cpu().eval()
example_input = torch.rand(1, 3, 400, 400) 
traced_model = torch.jit.trace(model, example_input)

class_labels = data.classes
model = ct.convert(traced_model,
                   inputs=[ct.ImageType(name='input_1', shape=example_input.shape)], 
                   classifier_config = ct.ClassifierConfig(class_labels))
model.save('MobileNetV2.mlmodel')

Translating MIL ==> MLModel Ops: 100%|██████████| 817/817 [00:00<00:00, 2250.89 ops/s]
